## Star formation function and the SFRD

In [ ]:
from functools import partial
import numpy as np
import h5py
from unyt import Msun, Myr, Angstrom, erg, s, Hz
from astropy.cosmology import LambdaCDM
import pandas as pd
from schwimmbad import MultiPool
from uncertainties import unumpy
from scipy import integrate

from synthesizer.grid import Grid
from synthesizer.particle import Galaxy
from synthesizer.instruments.filters import FilterCollection

from synthesizer.emission_models import PacmanEmission
from synthesizer.emission_models.attenuation import PowerLaw, Calzetti2000, GrainsWD01
from synthesizer.emissions.utils import (
    Ha,
    Hb, 
)
from synthesizer.conversions import lnu_to_absolute_mag

np.random.seed(680010)

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

from lmfit import Model

from utilities import calc_line_corr, calc_line_corr_from_Av, get_flares_LF

cosmo = LambdaCDM(Om0=0.307, Ode0=0.693, H0=67.77, Ob0=0.04825)
plt.style.use('styles/style.mplstyle')

In [ ]:
def Ha_to_SFR(Ha):
    
    sfr = 10**(-41.36) * Ha
    
    return sfr

def unc_schechter_fn(logL, logLstar, logphistar, alpha):
    
    exponent = (logL - logLstar) * (alpha+1)
    
    return logphistar + exponent + unumpy.log10(np.log(10) * np.e**(- 10**(logL - logLstar) ))

def schechter_fn(logL, logLstar, logphistar, alpha):
    
    exponent = (logL - logLstar) * (alpha+1)
    
    return logphistar + exponent + np.log10(np.log(10) * np.exp(- 10**(logL - logLstar) ))

def unc_schechter_log(logL, logLstar, logphistar, alpha):
    
    x = logL - logLstar
    phistar = 10**logphistar
    
    return unumpy.log(10) * phistar * (10 ** (x)) ** (alpha + 1) * np.e**(-10 ** x)

def fit_schechter(phi, logL):
    
    model = Model(schechter_fn)

    model.set_param_hint('alpha', min=-3.5, max=-0.5)
    model.set_param_hint('logLstar', min=0.5, max=3)
    model.set_param_hint('logphistar', min=-5, max=-1.5)    

    result = model.fit(phi, logL=logL, logLstar=2., logphistar=-2.5, alpha=-1.5, method='emcee', fit_kws={'steps':4000, 'burn':1000, 'nwalkers':200, 'thin':20})
    
    return result

def get_data(ii, tag):

    num = str(ii)

    if len(num) == 1:
        num =  '0'+num

    sim = "./data/flares_shared_data.hdf5"
    num = num+'/'

    with h5py.File(sim, 'r') as hf:
        Mstar   = np.array(hf[num+tag+'/Galaxy/Mstar_aperture'].get('30'), dtype = np.float32)*1e10
        SFR10   = np.array(hf[num+tag+'/Galaxy/SFR_aperture/30'].get('10Myr'), dtype = np.float32)
        
        MassweightedZ = np.array(hf[num+tag+'/Galaxy/Metallicity'].get('CurrentMassWeightedStellarZ'), dtype = np.float32)
        MFUVatt = np.array(hf[num+tag+'/Galaxy/BPASS_2.2.1/Chabrier300/Luminosity/DustModelI'].get('FUV'), dtype = np.float32) * erg / s / Hz
        MFUVint = np.array(hf[num+tag+'/Galaxy/BPASS_2.2.1/Chabrier300/Luminosity/Intrinsic'].get('FUV'), dtype = np.float32) * erg / s / Hz

        Av = np.array(hf[num+tag+'/Galaxy/BPASS_2.2.1/Chabrier300/Luminosity/DustModelI'].get('V'), dtype = np.float32) / np.array(hf[num+tag+'/Galaxy/BPASS_2.2.1/Chabrier300/Luminosity/Intrinsic'].get('V'), dtype = np.float32)
        Av = -2.5 * np.log10(Av)

        Halpha = np.array(hf[num+tag+'/Galaxy/BPASS_2.2.1/Chabrier300/Lines/DustModelI/HI6563'].get('Luminosity'), dtype = np.float64)
        Hbeta = np.array(hf[num+tag+'/Galaxy/BPASS_2.2.1/Chabrier300/Lines/DustModelI/HI4861'].get('Luminosity'), dtype = np.float64)
        Halphaint = np.array(hf[num+tag+'/Galaxy/BPASS_2.2.1/Chabrier300/Lines/Intrinsic/HI6563'].get('Luminosity'), dtype = np.float64)
        Hbetaint = np.array(hf[num+tag+'/Galaxy/BPASS_2.2.1/Chabrier300/Lines/Intrinsic/HI4861'].get('Luminosity'), dtype = np.float64)      


    ok = (Halphaint>0) * (lnu_to_absolute_mag(MFUVatt)<-17)

    return Mstar[ok], MFUVatt[ok], Halpha[ok], Hbeta[ok], Halphaint[ok], Hbetaint[ok], MassweightedZ[ok], MFUVint[ok], Av[ok], SFR10[ok]

In [ ]:
# Define the grid
grid_name = "bpass-2.2.1-bin_chabrier03-0.1,300.0_cloudy-c23.01-sps"
grid_dir = "./data/" 
grid = Grid(grid_name, grid_dir=grid_dir)

Zsun = 0.014
req_lines = {'H 1 6562.80A': 'Halpha',
             'H 1 4861.32A': 'Hbeta',
             'O 2 3726.03A,O 2 3728.81A': '[OII]3727,29',
             'O 3 4958.91A': '[OIII]4959',
             'O 3 5006.84A': '[OIII]5007',
             'Ne 3 3868.76A': 'NeIII3869'
}

tophats = {
    "UV1500": {"lam_eff": 1500*Angstrom, "lam_fwhm": 300*Angstrom},
    "UV2800": {"lam_eff": 2800*Angstrom, "lam_fwhm": 300*Angstrom},
}

filters = FilterCollection(
    filter_codes=[f'Generic/Johnson.V'],
    tophat_dict=tophats,
    new_lam=grid.lam
)

li = [
       Ha,
       Hb, 
   ]

lam_Ha = 6562.80 * Angstrom
lam_Hb = 4861.32 * Angstrom

In [ ]:
redshift = 6.
# Number of star particles
n = 100
ages = 10**np.random.uniform(0, np.log10(50), size=n) * Myr
Z = 10**np.random.uniform(-3.5, -2., size=n)
mass = 1e6*np.random.uniform(2, 5, n) * Msun 

print (r'log$_{10}$(M$_{\star}$/M$_{\odot}$): ', np.log10(np.sum(mass).to('Msun')))

# Dust distributions
max_tau = 7
mu, sigma = 0.1, 0.2
nsigma = np.arange(0,max_tau,2)
num_gals = len(nsigma)

# Property holders
logOH = np.zeros(4)
lum_int = np.zeros(4, dtype=object)
line_int = np.zeros(4, dtype=object)
sed_int = np.zeros(4, dtype=object)

line_emergent = np.zeros((4, num_gals), dtype=object)
sed_emergent = np.zeros((4, num_gals), dtype=object)
lum_att = np.zeros((4, num_gals))
att_curve = np.zeros((4, num_gals, len(grid.lam)))
Av = np.zeros((4, num_gals))
#intrinsic
Luv = np.zeros(4)
Halpha_frac_dust = np.zeros((4, num_gals))
Halpha_frac_Avcorr = np.zeros((4, num_gals))
Halpha_frac_balmercorr = np.zeros((4, num_gals))

# Vary everything
gal = Galaxy(redshift=redshift)
gal.load_stars(
    ages=ages,
    metallicities=Z,
    initial_masses=mass 
)
ok = gal.stars.ages<=10*Myr
Zmean = np.sum(gal.stars.initial_masses[ok] * gal.stars.metallicities[ok]) / np.sum(gal.stars.initial_masses[ok])
Agemean = np.sum(gal.stars.initial_masses[ok] * gal.stars.ages[ok]) / np.sum(gal.stars.initial_masses[ok])

print (Zmean, Agemean)

# Keep everything fixed
gal_const = Galaxy(redshift=redshift)
gal_const.load_stars(
    ages=np.ones(n) * Agemean,
    metallicities=np.ones(n) * Zmean,
    initial_masses=mass 
)

# Vary age
gal_vary_age = Galaxy(redshift=redshift)
gal_vary_age.load_stars(
    ages=ages,
    metallicities=np.ones(n) * Zmean,
    initial_masses=mass 
)

# Vary metallicity
gal_vary_Z = Galaxy(redshift=redshift)
gal_vary_Z.load_stars(
    ages=np.ones(n) * Agemean,
    metallicities=Z,
    initial_masses=mass 
)

all_gals = [gal_const, gal_vary_age, gal_vary_Z, gal]

tau_v = np.zeros((num_gals, n))
for ii in range(num_gals):
    tmp  = mu + np.random.normal(0, nsigma[ii], n)
    tmp[tmp<0] = 0.01
    tau_v[ii] = tmp

for jj, kk in enumerate(all_gals):
    print ("Galaxy number: ", jj)
    ok = kk.stars.ages<=10*Myr
    Zmean = np.sum(kk.stars.initial_masses[ok] * kk.stars.metallicities[ok]) / np.sum(kk.stars.initial_masses[ok])    
    logOH[jj] = 8.69 + np.log10(Zmean/Zsun)

    for ii in range(num_gals):

        emodel = PacmanEmission(
            grid=grid,
            tau_v=tau_v[ii],
            dust_curve=PowerLaw(),
            per_particle=True
        )   
    
        kk.stars.get_spectra(emission_model=emodel)    
        kk.stars.get_lines(li,
            emodel
        )        
        
        line_emergent[jj][ii] = kk.stars.lines['emergent']
        kk.stars.spectra['emergent'].get_fnu(cosmo=cosmo, z=redshift)   
        tmp = kk.stars.spectra['emergent'].get_photo_lnu(filters, verbose=False)
        lum_att[jj][ii] = tmp['Generic/Johnson.V']
        
        sed_emergent[jj][ii] = kk.stars.spectra['emergent']
        att_curve[jj][ii] = -2.5 * np.log10(kk.stars.spectra['emergent'].luminosity/kk.stars.spectra['intrinsic'].luminosity)

    line_int[jj] = kk.stars.lines['intrinsic']    
    lum_int[jj] = kk.stars.spectra['intrinsic'].get_photo_lnu(filters, verbose=False)['Generic/Johnson.V']    
    Luv[jj] = kk.stars.spectra['intrinsic'].get_photo_lnu(filters, verbose=False)['UV1500'] 
    sed_int[jj] = kk.stars.spectra['intrinsic']

    Av[jj] = -2.5 * np.log10(lum_att[jj]/lum_int[jj])
    
for jj, kk in enumerate(all_gals):
    for ii in range(num_gals):
        Halpha_frac_dust[jj][ii] = line_emergent[jj][ii]['Ha'].luminosity / line_int[jj]['Ha'].luminosity
        
        Halpha_frac_Avcorr[jj][ii] = calc_line_corr_from_Av(line_emergent[jj][ii]['Ha'].luminosity, lam_Ha, Av[jj][ii], slope=0) / line_int[jj]['Ha'].luminosity
        
        Halpha_frac_balmercorr[jj][ii] = calc_line_corr(line_emergent[jj][ii]['Ha'].luminosity, lam_Ha, line_emergent[jj][ii]['Ha'].luminosity/line_emergent[jj][ii]['Hb'].luminosity) / line_int[jj]['Ha'].luminosity
        
print (Av)

In [ ]:
# Star formation and metal enrichment history
title = [r'Varying only A$_{\mathrm{V}}$', 'Varying A$_{\mathrm{V}}$ and ages', 'Varying A$_{\mathrm{V}}$ and Z', 'Varying all']
for jj, kk in enumerate(all_gals):
    kk.stars.get_sfzh(log10ages=np.arange(6,7.81,0.05), log10metallicities=np.arange(-3.6,-1.9,0.05))
    fig, ax = kk.stars.plot_sfzh(show=False)
    ax.xaxis.label.set_size(14)
    ax.yaxis.label.set_size(14)
    for label in (ax.get_xticklabels() + ax.get_yticklabels()):
        label.set_fontsize(12)
    if jj>0:
        ax.set_ylabel('')
    fig.suptitle(title[jj], fontsize=13)
    fig.savefig(F'plots/sfzh_{jj}.png', bbox_inches='tight', dpi=300)
    plt.close()
    

In [ ]:
fig, axs = plt.subplots(nrows = 2, ncols = 1, figsize=(5, 8), sharex=True, sharey=False, facecolor='w', edgecolor='k')
axs = axs.ravel()

req_gal = 0

req_sed = sed_emergent[req_gal]
req_att_curve = att_curve[req_gal]
req_Av = Av[req_gal]

for ii in range(num_gals):
    color = plt.cm.tab10(ii) 
    ok = np.where(grid.lam<=1e4 * Angstrom)[0]
    
    axs[0].plot(grid.lam[ok]/1e4, req_sed[ii].lnu[ok], color=color, alpha=0.7)
    axs[1].plot(grid.lam[ok]/1e4, req_att_curve[ii][ok]/req_Av[ii], label=rF'$\sigma={nsigma[ii]}$ $(Av={np.round(Av[0][ii],2)})$', alpha=0.6, lw=2, color=color)    

axs[0].plot(grid.lam[ok]/1e4, sed_int[req_gal].lnu[ok], color='black', label='Intrinsic')

c = GrainsWD01(model='SMC')
axs[1].plot(grid.lam[ok]/1e4, c.get_tau(grid.lam[ok])/c.get_tau(5500*Angstrom), label='SMC', alpha=0.8, ls='dashed', lw=4, color='olive')

c = PowerLaw()
axs[1].plot(grid.lam[ok]/1e4, c.get_tau(grid.lam[ok])/c.get_tau(5500*Angstrom), label='Powerlaw, $\delta=-1$', alpha=0.6, ls='dotted', lw=4, color='olive')

c = Calzetti2000()
axs[1].plot(grid.lam[ok]/1e4, c.get_tau(grid.lam[ok])/c.get_tau(5500*Angstrom), label=r'Calzetti, $\delta=0$', alpha=0.6, ls='solid', lw=4, color='grey')

c = Calzetti2000(slope=0.5)
axs[1].plot(grid.lam[ok]/1e4, c.get_tau(grid.lam[ok])/c.get_tau(5500*Angstrom), label=r'Calzetti, $\delta=0.5$', alpha=0.6, ls='dashed', lw=4, color='grey')

for ax in axs:
    ax.grid(ls='dotted')
    ax.legend(frameon=False, fontsize=11) 
    ax.set_xlim(0.09,1)
    for label in (ax.get_xticklabels() + ax.get_yticklabels()):
        label.set_fontsize(13)
    
axs[0].set_ylabel(r'L$_{\nu}$/(erg/s/Hz)', fontsize=14)
axs[0].set_ylim(5e28, 2e30)
axs[0].set_yscale('log')

axs[1].set_xlabel(r'$\lambda_{\rm rest}$/${\rm \mu m}$', fontsize=14)
axs[1].set_ylim(0,6)
axs[1].set_ylabel(r'A$_{\lambda}$/A$_{V}$', fontsize=14)

fig.subplots_adjust(hspace=0,wspace=0)   
plt.savefig('plots/toy_att_curves.png', dpi=300, bbox_inches='tight')   
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize=(5,5), sharex=True, sharey=True, facecolor='w', edgecolor='k')
twinax = ax.twinx()

markers = ['s', 'H', '*', 'o']
custom = [Line2D([], [], marker=ii, markersize=8, color='dodgerblue', linestyle='None') for ii in markers]

for jj in range(4):    
    for ii in range(num_gals):
        color = plt.cm.tab10(ii)        
      
        # ax.scatter(Av[jj][ii], Halpha_frac_dust[jj][ii], color=color, s=30, marker=markers[jj], alpha=0.8, edgecolors='None')
        
        if jj==0:
            ax.scatter(Av[jj][ii], Halpha_frac_Avcorr[jj][ii], color=color, s=60, marker=markers[jj], alpha=0.5, label=rF'$\sigma={nsigma[ii]}$')
        else:
            ax.scatter(Av[jj][ii], Halpha_frac_Avcorr[jj][ii], color=color, s=60, marker=markers[jj], alpha=0.5)
            
        ax.scatter(Av[jj][ii], Halpha_frac_balmercorr[jj][ii], edgecolor=color, facecolor='None', s=100, marker=markers[jj])


ax.set_xlabel(r'A$_{V}$', fontsize=14)
ax.set_ylabel(r'SFR$_{{\rm dust\, corr}}$/SFR$_{\rm int}$', fontsize=14)
ax.set_ylim(0.55, 1.05)
ax.set_xlim(0., 0.75)
ax.grid(ls='dotted')
ax.legend(frameon=True, fontsize=11, ncols=2, loc='lower left') 
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(12)
twinax.set_yticks([])
twinax.set_yticklabels([])

twinax.legend(custom, [r'Varying only A$_{\mathrm{V}}$', 'Varying A$_{\mathrm{V}}$ and ages', 'Varying A$_{\mathrm{V}}$ and Z', 'Varying all'], frameon=False, fontsize=11, loc=(0.01,1.01), ncols=2, framealpha=0.4)
twinax.set_yticks([])
twinax.set_yticklabels([])

plt.savefig('plots/toy_sfr_frac.png', dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
h = 0.6777
vol = (4/3)*np.pi*(14/h)**3
quantiles = [0.16,0.50,0.84]
df = pd.read_csv('./data/weights_grid.txt')
weights = np.array(df['weights'])

sfrbins = np.arange(0.2, 2.91, 0.2)
bincen = (sfrbins[1:]+sfrbins[:-1])/2.
binwidth = sfrbins[1:] - sfrbins[:-1]

sfrbinslow = np.arange(0.5, 2.91, 0.2)
sfrbinshigh = np.arange(1.5, 2.91, 0.2)

tags = ['010_z005p000', '009_z006p000', '008_z007p000', '007_z008p000', '006_z009p000', '005_z010p000']

int_frac = np.zeros((2, len(tags)))
int_frac_err = np.zeros((2, len(tags)))

obsc_frac = np.zeros((2, len(tags)))
obsc_frac_err = np.zeros((2, len(tags)))

sfrd_frac_Av = np.zeros((2, len(tags)))
sfrd_frac_Av_err = np.zeros((2, len(tags)))

sfrd_frac_balmer = np.zeros((2, len(tags)))
sfrd_frac_balmer_err = np.zeros((2, len(tags)))

fig, axs = plt.subplots(nrows = 2, ncols = 3, figsize=(12, 6), sharex=True, sharey=True, facecolor='w', edgecolor='k')
axs = axs.ravel()

for ii, tag in enumerate(tags):
       
    z = float(tag[5:].replace('p','.'))

    func    = partial(get_data, tag=tag)
    pool    = MultiPool(processes=4)
    dat     = np.array(list(pool.map(func, np.arange(0,40))), dtype='object')
    pool.close()

    Mstar = np.log10(np.concatenate(dat[:,0]))
    MFUVatt = np.concatenate(dat[:,1])
    Halpha = np.concatenate(dat[:,2])
    Hbeta = np.concatenate(dat[:,3])
    Halphaint = np.concatenate(dat[:,4])
    Hbetaint = np.concatenate(dat[:,5])
    MassweightedZ = np.concatenate(dat[:,6])
    MFUVint = np.concatenate(dat[:,7])
    Av = np.concatenate(dat[:,8])
    SFR10 = np.concatenate(dat[:,9])
    
    Halpha_Av = calc_line_corr_from_Av(Halpha * erg/s, lam_Ha, Av, slope=0).to_value()
    Halpha_balmer =  calc_line_corr(Halpha * erg/s, lam_Ha, Halpha/Hbeta, slope=0).to_value() 
    
    sfr_int = np.log10(Ha_to_SFR(Halphaint))
    sfr_dust = np.log10(Ha_to_SFR(Halpha))
    sfr_Av = np.log10(Ha_to_SFR(Halpha_Av))
    sfr_balmer = np.log10(Ha_to_SFR(Halpha_balmer))
    
    sfr10_int = np.log10(SFR10)
    
    # f, ax = plt.subplots(nrows = 1, ncols = 1, figsize=(3, 3))
    # ax.scatter(sfr10_int, np.log10(MassweightedZ), s=1)
    # ax.set_ylim(-5,-1.5)
    # ax.set_xlim(0,3)
    # f.savefig(F'app_plots/sfr_Z_z{z}.png')
    # f.close()
    
    tot_gals = len(Mstar)
    
    n = np.array([])
    for jj in range(40):
        n = np.append(n, len(dat[jj][0]))

    #Make the SFR distribution function, integrate the different ratio to get the recovered fraction  
   
    #Intrinsic
    hist, M, err = get_flares_LF(sfr_int, weights, sfrbins, n)
    ok = np.where(hist>=5)[0]
    phi, phierr = M/(binwidth*vol), err/(binwidth*vol)
    axs[ii].errorbar(bincen, np.log10(phi), yerr=[np.log10(phi)-np.log10(phi-phierr), np.log10(phi+phierr)-np.log10(phi)], ls='None', marker='s', color='black', label='Intrinsic', alpha=0.5, markersize=7)
    # axs[ii].fill_between(bincen[ok], np.log10(phi[ok]-phierr[ok]), np.log10(phi[ok]+phierr[ok]), alpha=0.25, color='black')
    
    ok = np.isfinite(np.log10(phi)) * np.isfinite(bincen)
    result = fit_schechter(np.log10(phi[ok]), bincen[ok])
    print (z, 'int', result.fit_report())
    fit = np.array([*result.best_values.values()])
    fiterr = np.array(result.flatchain.std())
    f = unc_schechter_fn(sfrbins, unumpy.uarray(fit[0], fiterr[0]), unumpy.uarray(fit[1], fiterr[1]), unumpy.uarray(fit[2], fiterr[2]))
    axs[ii].plot(sfrbins, unumpy.nominal_values(f), lw=4, alpha=0.4, ls='solid', label='Intrinsic-fit', color='black')
    
    f = unc_schechter_log(sfrbinslow, unumpy.uarray(fit[0], fiterr[0]), unumpy.uarray(fit[1], fiterr[1]), unumpy.uarray(fit[2], fiterr[2]))
    tot = integrate.simpson(f, sfrbinslow) 
    
    f = unc_schechter_log(sfrbinshigh, unumpy.uarray(fit[0], fiterr[0]), unumpy.uarray(fit[1], fiterr[1]), unumpy.uarray(fit[2], fiterr[2]))
    tothigh = integrate.simpson(f, sfrbinshigh)
    
    
    # SFR (10 Myr)
    hist, M, err = get_flares_LF(sfr10_int, weights, sfrbins, n)
    ok = np.where(hist>=5)[0]
    phi, phierr = M/(binwidth*vol), err/(binwidth*vol)
    axs[ii].errorbar(bincen, np.log10(phi), yerr=[np.log10(phi)-np.log10(phi-phierr), np.log10(phi+phierr)-np.log10(phi)], ls='None', marker='s', color='grey', label='SFR (10Myr)', alpha=0.9, markersize=4)
    
    ok = np.isfinite(np.log10(phi)) * np.isfinite(bincen)
    result = fit_schechter(np.log10(phi[ok]), bincen[ok])
    # print (z, 'int', result.fit_report())
    fit = np.array([*result.best_values.values()])
    fiterr = np.array(result.flatchain.std())
    f = unc_schechter_fn(sfrbins, unumpy.uarray(fit[0], fiterr[0]), unumpy.uarray(fit[1], fiterr[1]), unumpy.uarray(fit[2], fiterr[2]))
    
    f = unc_schechter_log(sfrbinslow, unumpy.uarray(fit[0], fiterr[0]), unumpy.uarray(fit[1], fiterr[1]), unumpy.uarray(fit[2], fiterr[2]))
    tmp = integrate.simpson(f, sfrbinslow) / tot    
    int_frac[0][ii] = unumpy.nominal_values(tmp)
    int_frac_err[0][ii] = unumpy.std_devs(tmp)
    
    f = unc_schechter_log(sfrbinshigh, unumpy.uarray(fit[0], fiterr[0]), unumpy.uarray(fit[1], fiterr[1]), unumpy.uarray(fit[2], fiterr[2]))
    tmp = integrate.simpson(f, sfrbinshigh) / tothigh
    int_frac[1][ii] = unumpy.nominal_values(tmp)
    int_frac_err[1][ii] = unumpy.std_devs(tmp)
    
    
    hist, M, err = get_flares_LF(sfr_dust, weights, sfrbins, n)
    ok = np.where(hist>=5)[0]
    phi, phierr = M/(binwidth*vol), err/(binwidth*vol)
    # axs[ii].plot(bincen, np.log10(phi), lw=4, alpha=0.4, ls='dashed', label='Observed', color='black')
    ok = np.isfinite(np.log10(phi)) * np.isfinite(bincen)
    result = fit_schechter(np.log10(phi[ok]), bincen[ok])
    print (z, 'obs', result.fit_report())
    fit = np.array([*result.best_values.values()])
    fiterr = np.array(result.flatchain.std())
    f = unc_schechter_fn(sfrbins, unumpy.uarray(fit[0], fiterr[0]), unumpy.uarray(fit[1], fiterr[1]), unumpy.uarray(fit[2], fiterr[2]))   
    axs[ii].plot(sfrbins, unumpy.nominal_values(f), lw=4, alpha=0.4, ls='dashed', label='Unobscured-fit', color='black') 
    
    f = unc_schechter_log(sfrbinslow, unumpy.uarray(fit[0], fiterr[0]), unumpy.uarray(fit[1], fiterr[1]), unumpy.uarray(fit[2], fiterr[2]))   
    tmp = integrate.simpson(f, sfrbinslow) / tot    
    obsc_frac[0][ii] = unumpy.nominal_values(tmp)
    obsc_frac_err[0][ii] = unumpy.std_devs(tmp)
    f = unc_schechter_log(sfrbinshigh, unumpy.uarray(fit[0], fiterr[0]), unumpy.uarray(fit[1], fiterr[1]), unumpy.uarray(fit[2], fiterr[2]))
    tmp = integrate.simpson(f, sfrbinshigh) / tothigh
    obsc_frac[1][ii] = unumpy.nominal_values(tmp)
    obsc_frac_err[1][ii] = unumpy.std_devs(tmp)

    
    
    hist, M, err = get_flares_LF(sfr_Av, weights, sfrbins, n)
    ok = np.where(hist>=5)[0]
    phi, phierr = M/(binwidth*vol), err/(binwidth*vol)

    ok = np.isfinite(np.log10(phi)) * np.isfinite(bincen)
    result = fit_schechter(np.log10(phi[ok]), bincen[ok])
    print (z, 'Av', result.fit_report())
    fit = np.array([*result.best_values.values()])
    fiterr = np.array(result.flatchain.std())
    f = unc_schechter_fn(sfrbins, unumpy.uarray(fit[0], fiterr[0]), unumpy.uarray(fit[1], fiterr[1]), unumpy.uarray(fit[2], fiterr[2]))  
    axs[ii].plot(sfrbins, unumpy.nominal_values(f), lw=4, alpha=0.4, ls='solid', label='Av corrected', color='blue')  
    
    f = unc_schechter_log(sfrbinslow, unumpy.uarray(fit[0], fiterr[0]), unumpy.uarray(fit[1], fiterr[1]), unumpy.uarray(fit[2], fiterr[2]))  
    tmp = integrate.simpson(f, sfrbinslow) / tot
    sfrd_frac_Av[0][ii] = unumpy.nominal_values(tmp)
    sfrd_frac_Av_err[0][ii] = unumpy.std_devs(tmp)
    f = unc_schechter_log(sfrbinshigh, unumpy.uarray(fit[0], fiterr[0]), unumpy.uarray(fit[1], fiterr[1]), unumpy.uarray(fit[2], fiterr[2]))
    tmp = integrate.simpson(f, sfrbinshigh) / tothigh
    sfrd_frac_Av[1][ii] = unumpy.nominal_values(tmp)
    sfrd_frac_Av_err[1][ii] = unumpy.std_devs(tmp)

    
    
    hist, M, err = get_flares_LF(sfr_balmer, weights, sfrbins, n)
    ok = np.where(hist>=5)[0]
    phi, phierr = M/(binwidth*vol), err/(binwidth*vol)

    ok = np.isfinite(np.log10(phi)) * np.isfinite(bincen)
    result = fit_schechter(np.log10(phi[ok]), bincen[ok])
    print (z, 'balmer', result.fit_report())
    fit = np.array([*result.best_values.values()])
    fiterr = np.array(result.flatchain.std())
    f = unc_schechter_fn(sfrbins, unumpy.uarray(fit[0], fiterr[0]), unumpy.uarray(fit[1], fiterr[1]), unumpy.uarray(fit[2], fiterr[2]))    
    axs[ii].plot(sfrbins, unumpy.nominal_values(f), lw=4, alpha=0.4, ls='solid', label='Balmer corrected', color='brown')  
    
    f = unc_schechter_log(sfrbinslow, unumpy.uarray(fit[0], fiterr[0]), unumpy.uarray(fit[1], fiterr[1]), unumpy.uarray(fit[2], fiterr[2]))    
    tmp = integrate.simpson(f, sfrbinslow) / tot
    sfrd_frac_balmer[0][ii] = unumpy.nominal_values(tmp)
    sfrd_frac_balmer_err[0][ii] = unumpy.std_devs(tmp)
    f = unc_schechter_log(sfrbinshigh, unumpy.uarray(fit[0], fiterr[0]), unumpy.uarray(fit[1], fiterr[1]), unumpy.uarray(fit[2], fiterr[2]))
    tmp = integrate.simpson(f, sfrbinshigh) / tothigh
    sfrd_frac_balmer[1][ii] = unumpy.nominal_values(tmp)
    sfrd_frac_balmer_err[1][ii] = unumpy.std_devs(tmp)
   
    axs[ii].text(2.1, -3.4, r'$z = {}$'.format(z), fontsize = 13)
    
for ax in axs:
    ax.grid(True, alpha=0.6, ls='dotted')
    ax.set_xlim(0., 2.8)
    ax.set_ylim(-7.5, -1.7)
    ax.minorticks_on()
    ax.tick_params(axis='x', which='minor', direction='in')
    ax.tick_params(axis='y', which='minor', direction='in')
    for label in (ax.get_xticklabels() + ax.get_yticklabels()):
        label.set_fontsize(12)

axs[0].legend(frameon=False, fontsize=12)
axs[4].set_xlabel(r'log$_{10}$($\mathrm{SFR}$/(M$_{\odot}$/yr))', fontsize=15)

fig.subplots_adjust(bottom=0.12, wspace=0, hspace=0)

fig.text(0.065, 0.5, r'log$_{10}$($\Phi$/($\mathrm{cMpc}^{-3}\mathrm{dex}^{-1}$))', va='center', rotation='vertical', fontsize=15)

plt.savefig('plots/sfr_function.png', dpi=300, bbox_inches='tight')

In [ ]:
fig, axs = plt.subplots(nrows = 1, ncols = 2, figsize=(9, 5), sharex=True, sharey=True, facecolor='w', edgecolor='k')
axs = axs.ravel()

zs = np.arange(5,11,1)

axs[0].plot(zs, int_frac[0], color='black', label='SFR (10 Myr)', ls='dotted', lw=2, alpha=0.5)

axs[0].plot(zs, obsc_frac[0], color='black', label='Unobscured', ls='dashed', lw=4, alpha=0.5)

axs[0].plot(zs, sfrd_frac_Av[0], color='blue', label='Av corrected', lw=4, alpha=0.5)

axs[0].plot(zs, sfrd_frac_balmer[0], color='brown', label='Balmer corrected', lw=4, alpha=0.5, zorder=10)


axs[1].plot(zs, int_frac[1], color='black', ls='dotted', lw=2, alpha=0.5)

axs[1].plot(zs, obsc_frac[1], color='black', ls='dashed', lw=4, alpha=0.5)

axs[1].plot(zs, sfrd_frac_Av[1], color='blue', lw=4, alpha=0.5)

axs[1].plot(zs, sfrd_frac_balmer[1], color='brown', lw=4, alpha=0.5)


axs[0].legend(fontsize=12, frameon=True)

for ax in axs:
    ax.axhline(y=1.0, color='black', lw=2)
    ax.grid(ls='dotted')
    ax.set_ylim(0.2, 1.15)
    ax.set_yticks(np.arange(0.2, 1.16, 0.1))
    ax.set_xlabel(r'$z$', fontsize=16)
    for label in (ax.get_xticklabels() + ax.get_yticklabels()):
        label.set_fontsize(12)

axs[0].set_ylabel('Fraction', fontsize=14)
axs[0].set_title(r'SFR $>$ 3 M$_{\odot}$/yr', fontsize=14)   
axs[1].set_title(r'SFR $>$ 30 M$_{\odot}$/yr', fontsize=14)   

fig.subplots_adjust(hspace=0,wspace=0)
plt.savefig('plots/sfrd_frac.png', dpi=300, bbox_inches='tight')
